In [2]:
import json
import requests
import pandas as pd
from datetime import datetime
from espn_api.football import League

In [2]:
LEAGUE_ID = 48899928
YEAR = datetime.today().year
league = League(league_id=LEAGUE_ID, year=YEAR)

# Weekly Matchup

In [3]:
box_scores = league.box_scores(week=league.currentMatchupPeriod)
box_score = box_scores[0]

In [4]:
def get_team_lineup(lineup_list: list) -> list:
    """
    Gathers certain fields from either
    `BoxScore.home_lineup` or `BoxScore.away_lineup`
    and returns a single dictionary.

    Arguments:
        lineup_list (list): List of `espn_api.football.box_player.BoxPlayer`.

    Returns:
        list: List of dictionaries of players and their attributes.

    """

    team = []
    for player in get_team_lineup:
        data = {}
        data["injuryStatus"] = (
            "-" if player.injuryStatus == "ACTIVE" else player.injuryStatus
        )
        data["name"] = player.name
        data["playerId"] = player.playerId
        data["points"] = player.points
        data["projected_points"] = player.projected_points
        data["position"] = (
            "FLEX" if player.slot_position == "RB/WR/TE" else player.slot_position
        )
        team.append(data)
    return team

In [5]:
box_score.away_team.logo_url

'https://i.imgur.com/lS8cwSg.jpg'

In [6]:
box_score.home_team.wins

0

In [7]:
matchups = []
for matchup in box_scores:
    data = {}

    away_team = get_team_lineup(box_score.away_lineup)
    data["away_team"] = away_team
    data["away_score"] = box_score.away_score
    data["away_projected"] = box_score.away_projected

    home_team = get_team_lineup(box_score.home_lineup)
    data["home_team"] = home_team
    data["home_score"] = box_score.home_score
    data["home_projected"] = box_score.home_projected

    data["isPlayoffGame"] = str(box_score.matchup_type).lower() == "none"

    team, score = league.least_scored_week()
    data["lowest_scoring_team"] = team.team_name
    data["lowest_scoring_team_score"] = score

    team, score = league.top_scored_week()
    data["highest_scoring_team"] = team.team_name
    data["highest_scoring_team_score"] = score

    matchups.append(data)

TypeError: 'function' object is not iterable

# Top Free Agents

In [8]:
available_free_agents = league.free_agents()

In [9]:
free_agents = []
for player in available_free_agents:
    free_agent = {}
    free_agent["name"] = player.name
    free_agent["points"] = player.points
    free_agent["projectedPoints"] = player.projected_points
    free_agent["injuryStatus"] = (
        "-"
        if player.injuryStatus == "ACTIVE" or player.injuryStatus == []
        else player.injuryStatus
    )
    free_agent["position"] = player.position
    free_agent["team"] = player.proTeam
    free_agent["opponent"] = player.pro_opponent
    free_agents.append(free_agent)

free_agents = pd.DataFrame(free_agents)
top_projected_free_agents = (
    free_agents.groupby(["position"])
    .apply(lambda x: x.sort_values(["projectedPoints"], ascending=False))
    .reset_index(drop=True)
)
top_five_projected_free_agents = top_projected_free_agents.groupby("position").head(5)

top_scoring_free_agents = (
    free_agents.groupby(["position"])
    .apply(lambda x: x.sort_values(["points"], ascending=False))
    .reset_index(drop=True)
)
top_five_scoring_free_agents = top_scoring_free_agents.groupby("position").head(5)
top_five_projected_free_agents = top_five_projected_free_agents.to_dict(
    orient="records"
)
top_five_scoring_free_agents = top_five_scoring_free_agents.to_dict(orient="records")

# Standings

In [10]:
standings = []
for idx, team in enumerate(league.standings()):
    standing = {}
    standing["standing"] = team.standing
    standing["team_name"] = team.team_name
    standing["wins"] = team.wins
    standing["losses"] = team.losses
    standing["ties"] = team.ties
    standing["logo_url"] = team.logo_url
    standing["playoff_pct"] = team.playoff_pct
    standing["points_for"] = team.points_for
    standing["points_against"] = team.points_against
    standings.append(standing)

standings = pd.DataFrame(standings)
standings["standing"] = standings["standing"].astype(int)
standings.sort_values(by="standing", ascending=True, inplace=True)
standings["standing"] = standings["standing"].astype(str)
standings = standings.to_dict(orient="records")

# Power Rankings

In [11]:
power_rankings = []
for score, team in league.power_rankings():
    power_ranking = {}
    power_ranking["power_ranking"] = score
    power_ranking["team_name"] = team.team_name
    power_ranking["wins"] = team.wins
    power_ranking["losses"] = team.losses
    power_ranking["ties"] = team.ties
    power_ranking["logo_url"] = team.logo_url
    power_ranking["playoff_pct"] = team.playoff_pct
    power_ranking["points_for"] = team.points_for
    power_ranking["points_against"] = team.points_against
    power_rankings.append(power_ranking)

power_rankings = pd.DataFrame(power_rankings)
power_rankings["power_ranking"] = power_rankings["power_ranking"].astype(float)
power_rankings.sort_values(by="power_ranking", ascending=False, inplace=True)
power_rankings["power_ranking"] = power_rankings["power_ranking"].astype(str)
power_rankings = power_rankings.to_dict(orient="records")

In [12]:
import pandas as pd

In [13]:
injury_statuses = [
    "ACTIVE",
    "NORMAL",
    "QUESTIONABLE",
    "OUT",
    []
]

In [14]:
for is_ in injury_statuses:
    if is_ in ['ACTIVE', 'NORMAL', []]:
        print("new is: ", "-")
    elif is_ == "QUESTIONABLE":
        print("new is: ", "Q")
    else:
        print("new is: ", "O")

new is:  -
new is:  -
new is:  Q
new is:  O


# ESPN v3

statSourceId

In [3]:
base_url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/2022/segments/0/leagues/1150587'
roster_url = base_url + '?view=mRoster'

In [6]:
response = requests.get(roster_url)
if response.status_code == 200:
    data = response.json()
response.status_code

200

In [7]:
data.keys()

dict_keys(['draftDetail', 'gameId', 'id', 'scoringPeriodId', 'seasonId', 'segmentId', 'status', 'teams'])

In [16]:
team = data['teams'][0]

In [17]:
team.keys()

dict_keys(['id', 'roster'])

In [19]:
team['roster'].keys()

dict_keys(['appliedStatTotal', 'entries', 'tradeReservedEntries'])

In [23]:
team['roster']['entries'][0].keys()

dict_keys(['acquisitionDate', 'acquisitionType', 'injuryStatus', 'lineupSlotId', 'pendingTransactionIds', 'playerId', 'playerPoolEntry', 'status'])

In [24]:
team['roster']['entries'][0]['injuryStatus']

'NORMAL'

In [25]:
team['roster']['entries'][0]['lineupSlotId']

2

In [27]:
team['roster']['entries'][0]['playerPoolEntry'].keys()

dict_keys(['appliedStatTotal', 'id', 'keeperValue', 'keeperValueFuture', 'lineupLocked', 'onTeamId', 'player', 'ratings', 'rosterLocked', 'status', 'tradeLocked'])

In [35]:
team['roster']['entries'][0]['playerPoolEntry']['player'].keys()

dict_keys(['active', 'defaultPositionId', 'draftRanksByRankType', 'droppable', 'eligibleSlots', 'firstName', 'fullName', 'id', 'injured', 'injuryStatus', 'lastName', 'lastNewsDate', 'lastVideoDate', 'ownership', 'proTeamId', 'rankings', 'seasonOutlook', 'stats', 'universeId'])

In [50]:
team['roster']['entries'][0]['playerPoolEntry']['player']['stats'][0].keys()

dict_keys(['appliedAverage', 'appliedStats', 'appliedTotal', 'externalId', 'id', 'proTeamId', 'scoringPeriodId', 'seasonId', 'statSourceId', 'statSplitTypeId', 'stats'])

In [54]:
stats = team['roster']['entries'][0]['playerPoolEntry']['player']['stats']

In [81]:
for idx, stat in enumerate(stats):
    
    print('idx: {} | {} | {} | {}'.format(idx, stat['statSourceId'], stat['seasonId'], stat['scoringPeriodId']))

idx: 0 | 0 | 2021 | 0
idx: 1 | 0 | 2022 | 0
idx: 2 | 1 | 2021 | 0
idx: 3 | 1 | 2022 | 0
idx: 4 | 1 | 2022 | 1


In [89]:
newDict = dict(filter(lambda elem: (elem['seasonId'] == 2022 and elem['scoringPeriodId'] == 1), stats))

ValueError: dictionary update sequence element #0 has length 10; 2 is required

In [110]:
for stat in stats:
    if str(stat['scoringPeriodId']) == '1' and str(stat['seasonId']) == '2022':
        if str(stat['statSourceId']) == '0':
            points = stat
        elif str(stat['statSourceId'] == '1'):
            projected_points = stat

In [112]:
projected_points

{'appliedStats': {'53': 0.89646844,
  '72': -0.169704152,
  '24': 9.873019314,
  '25': 4.678253382,
  '26': 0.0482,
  '42': 1.374861863,
  '43': 0.370606632,
  '44': 0.006269136,
  '63': 0.004794},
 'appliedTotal': 17.082768615000003,
 'externalId': '20221',
 'id': '1120221',
 'proTeamId': 0,
 'scoringPeriodId': 1,
 'seasonId': 2022,
 'statSourceId': 1,
 'statSplitTypeId': 1,
 'stats': {'23': 23.30545644,
  '24': 98.73019314,
  '25': 0.779708897,
  '26': 0.0241,
  '27': 19.0,
  '28': 9.0,
  '29': 4.0,
  '30': 3.0,
  '31': 1.0,
  '33': 4.0,
  '34': 2.0,
  '35': 0.052,
  '36': 0.0364,
  '37': 0.427,
  '38': 0.0146,
  '39': 4.236355268,
  '40': 98.73019314,
  '42': 13.74861863,
  '43': 0.061767772,
  '44': 0.003134568,
  '45': 0.003039834,
  '46': 0.001986532,
  '47': 2.0,
  '48': 1.0,
  '53': 1.79293688,
  '56': 0.005182992,
  '57': 0.00016,
  '58': 2.322907929,
  '60': 7.66821118,
  '61': 13.74861863,
  '62': 0.027281348,
  '63': 0.000799,
  '66': 0.169,
  '67': 0.01297571,
  '68': 0.18

In [79]:
for l in list(stats[0].keys()):
    print(l)

appliedAverage
appliedStats
appliedTotal
externalId
id
proTeamId
scoringPeriodId
seasonId
statSourceId
statSplitTypeId
stats


In [74]:
sorted(keys)

['appliedAverage',
 'appliedStats',
 'appliedTotal',
 'externalId',
 'id',
 'proTeamId',
 'scoringPeriodId',
 'seasonId',
 'statSourceId',
 'statSplitTypeId',
 'stats']

In [77]:
stats[4]

{'appliedStats': {'53': 0.89646844,
  '72': -0.169704152,
  '24': 9.873019314,
  '25': 4.678253382,
  '26': 0.0482,
  '42': 1.374861863,
  '43': 0.370606632,
  '44': 0.006269136,
  '63': 0.004794},
 'appliedTotal': 17.082768615000003,
 'externalId': '20221',
 'id': '1120221',
 'proTeamId': 0,
 'scoringPeriodId': 1,
 'seasonId': 2022,
 'statSourceId': 1,
 'statSplitTypeId': 1,
 'stats': {'23': 23.30545644,
  '24': 98.73019314,
  '25': 0.779708897,
  '26': 0.0241,
  '27': 19.0,
  '28': 9.0,
  '29': 4.0,
  '30': 3.0,
  '31': 1.0,
  '33': 4.0,
  '34': 2.0,
  '35': 0.052,
  '36': 0.0364,
  '37': 0.427,
  '38': 0.0146,
  '39': 4.236355268,
  '40': 98.73019314,
  '42': 13.74861863,
  '43': 0.061767772,
  '44': 0.003134568,
  '45': 0.003039834,
  '46': 0.001986532,
  '47': 2.0,
  '48': 1.0,
  '53': 1.79293688,
  '56': 0.005182992,
  '57': 0.00016,
  '58': 2.322907929,
  '60': 7.66821118,
  '61': 13.74861863,
  '62': 0.027281348,
  '63': 0.000799,
  '66': 0.169,
  '67': 0.01297571,
  '68': 0.18